# Command Line Execution with suspicious URL and AppData Strings
Detects a suspicious command line execution that includes an URL and AppData string in the command line parameters as used by several droppers (js/vbs > powershell)

## Rule Content
```
- title: Command Line Execution with suspicious URL and AppData Strings
  id: 1ac8666b-046f-4201-8aba-1951aaec03a3
  status: experimental
  description: Detects a suspicious command line execution that includes an URL and
    AppData string in the command line parameters as used by several droppers (js/vbs
    > powershell)
  references:
  - https://www.hybrid-analysis.com/sample/3a1f01206684410dbe8f1900bbeaaa543adfcd07368ba646b499fa5274b9edf6?environmentId=100
  - https://www.hybrid-analysis.com/sample/f16c729aad5c74f19784a24257236a8bbe27f7cdc4a89806031ec7f1bebbd475?environmentId=100
  author: Florian Roth
  tags:
  - attack.execution
  - attack.t1059
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - cmd.exe /c *http://*%AppData%
      - cmd.exe /c *https://*%AppData%
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - High
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(cmd.exe\ \/c\ *http\:\/\/*%AppData% OR cmd.exe\ \/c\ *https\:\/\/*%AppData%)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()